In [2]:
import pandas as pd
import numpy as np

In [3]:
from collections import Counter

class convert2vec:

    def fit_texts(self, words, num_words):
        self.num_words = num_words
        te2 = []
        for i in range(len(words)):
            for j in range(len(words[i])):
                te2.append(words[i][j])
        q = Counter(te2)
        le = q.most_common(num_words - 1)
        temp = le[0:num_words - 1]
        self.words = []
        for item in temp:
            self.words.append(item[0])
        #print(self.words)

    def word2vector(self, words):
        vectors = []
        for i in range(len(words)):
            temp = []
            for word in words[i]:
                te = [0 for i in range(self.num_words)]
                te[self.num_words - 1] = 1
                for k in range(len(self.words)):
                    if word == self.words[k]:
                        te[k] = 1
                        te[self.num_words - 1] = 0
                        break
                temp.append(te)
            vectors.append(temp)
        return vectors

In [4]:
data1 = pd.read_csv('../input/spam.csv', encoding = 'latin-1')
list = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']
data1 = data1.drop(list, axis=1)

data2 = pd.ExcelFile('../input/indiandataset.xls')
data2 = data2.parse(0)
data2.head()

data = pd.concat([data1, data2])

In [5]:
from keras.preprocessing.text import text_to_word_sequence
temp = []
for row in data.itertuples():
    rev = str(row[2])
    rev = text_to_word_sequence(rev)
    temp.append(rev)

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
vec = convert2vec()
vec.fit_texts(temp, 3000)
data_vec = vec.word2vector(temp)

In [7]:
empty = [0 for i in range(3000)]
le = len(data_vec)
temp2 = [[empty for i in range(200)] for j in range(le)]

for i in range(le):
    for j in range(len(data_vec[i]) - 1,-1,-1):
        temp2[i][199 - (len(data_vec[i]) - 1 - j)] = data_vec[i][j];

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(temp2, data['label'], test_size = 0.30)

Y_train = pd.get_dummies(Y_train).values

In [9]:
# add anything to delete
del(temp)
del(data['msg'])
del(data_vec)
del(vec)
del(temp2)

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

embed_dim = 128
lstm_out = 196


model = Sequential()
model.add(Dense(300, activation = 'relu', input_shape = (200, 3000)))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(
    loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(196, dropout=0.2, recurrent_dropout=0.2)`
  # Remove the CWD from sys.path while we load stuff.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 200, 300)          900300    
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               389648    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 394       
Total params: 1,290,342
Trainable params: 1,290,342
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
X_train = np.array(X_train, dtype=np.uint8)

In [12]:
model.fit(X_train, Y_train, epochs = 5,batch_size = 20)

Epoch 1/5
7117/7117 [==============================] - 503s 71ms/step - loss: 0.2372 - acc: 0.9025
Epoch 2/5
7117/7117 [==============================] - 473s 66ms/step - loss: 0.0797 - acc: 0.9706
Epoch 3/5
7117/7117 [==============================] - 507s 71ms/step - loss: 0.0529 - acc: 0.9823
Epoch 4/5
7117/7117 [==============================] - 476s 67ms/step - loss: 0.0397 - acc: 0.9874
Epoch 5/5
7117/7117 [==============================] - 452s 63ms/step - loss: 0.0296 - acc: 0.9913


In [13]:
del(X_train)

In [14]:
X_test = np.array(X_test, dtype=np.uint8)

In [15]:
Y_pr = model.predict(X_test)

Y_pred = []

for i in range(len(Y_pr)):
    if Y_pr[i][0] >= Y_pr[i][1]:
        Y_pred.append('ham')
    else:
        Y_pred.append('spam')
        
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, Y_pred)  
print(cm)

[[2384   59]
 [  35  573]]
